In [8]:
import  torch
import  append_path
from    deep_numerical.numerical.solvers    import  FastSM_Boltzmann_VHS
from    deep_numerical.layers               import  FourierBoltzmannLayer
from    deep_numerical.numerical.distribution   import  bkw
from    deep_numerical.utils                import  *

In [25]:
dim, res, v_max = 2, 64, 5.0
vhs_coeff, vhs_alpha = 1.0, 0.0

v_grid = velocity_grid(dim, res, v_max)
numerical_solver = FastSM_Boltzmann_VHS(
    dimension   = dim,
    v_num_grid  = res,
    v_max       = v_max,
    vhs_coeff   = vhs_coeff,
    vhs_alpha   = vhs_alpha,
    dtype       = torch.double,
)
neural_solver = FourierBoltzmannLayer(
    dimension   = dim,
    degree      = res//2,
    n_weights   = numerical_solver.approximation_level,
    dtype_str   = "double",
)

In [26]:
numerical_solver.fsm_scale.shape, numerical_solver.fsm_phase_1.shape, numerical_solver.fsm_phase_2.shape, numerical_solver.kernel_diag.shape

(torch.Size([64, 64, 1, 64, 1]),
 torch.Size([64, 64, 1, 64, 21]),
 torch.Size([64, 64, 1, 64, 21]),
 torch.Size([64, 64, 1]))

In [27]:
neural_solver.params_alpha.shape, neural_solver.params_beta.shape, neural_solver.params_gamma.shape, neural_solver.params_diag.shape

(torch.Size([64, 64, 1, 1344]),
 torch.Size([64, 64, 1, 1344]),
 torch.Size([64, 64, 1, 1344]),
 torch.Size([64, 64, 1]))

In [28]:
neural_solver.params_alpha.data = numerical_solver.fsm_scale.repeat(1,1,1,1,21).reshape(neural_solver.params_alpha.shape)
neural_solver.params_beta.data = numerical_solver.fsm_phase_1.reshape(neural_solver.params_beta.shape)
neural_solver.params_gamma.data = numerical_solver.fsm_phase_2.reshape(neural_solver.params_gamma.shape)
neural_solver.params_diag.data = numerical_solver.kernel_diag.reshape(neural_solver.params_diag.shape)

In [29]:
f = bkw(torch.zeros((1,)), v_grid, vhs_coeff, 0.5)
print(f.shape)
fft_dim = tuple([-(2+k) for k in reversed(range(dim))])
fft_norm = 'forward'
f_fft: torch.Tensor = torch.fft.fftn(f, dim=fft_dim, norm=fft_norm)

----------[ BKW solution ]----------
* coeff_ext: 0.5
* coeff_int: 0.7853981633974483
------------------------------------
torch.Size([1, 1, 1, 1, 64, 64, 1])


In [ ]:
_sl = (slice(None), *(0 for _ in range(dim)))
g_fft_1 = numerical_solver.compute_loss_fft(None, f_fft)[*_sl]
g_fft_2 = neural_solver.compute_loss_fft(f_fft[*_sl])
print(g_fft_1.shape, g_fft_2.shape)
print((g_fft_2-g_fft_1).norm())

torch.Size([1, 1, 64, 64, 1]) torch.Size([1, 1, 64, 64, 1])
tensor(0.1558, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>)
